<a href="https://colab.research.google.com/github/rzl-ds/gu511/blob/master/008_iam.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg"/>
</a>

# `aws` identity access management (`iam`)

I like to say that if you're having a problem with `aws`, it's probably going to get fixed on the `iam` console.

the most common problem area for new users of cloud services -- and *especially* `aws` -- is permissions and security. in fact, the task can be confusing and daunting enough that entire companies have sprung up offering to take care of these issues for a fee. given that this is already a full time job for experienced IT staffers in every organization with shared resources (*e.g.* any company or university), this is perhaps not surprising.

it's easy to take that role for granted. I think learning some *basics* about what they do will not only allow you to appreciate what they're doing a bit more, it will give you the ability to think a bit more like they do, to diagnose and solve problems like they do, and to help communicate with them when help is needed.

plus, you just have to figure it out. it may get easier over time, but that doesn't matter right now!

with that set of "this is hard" caveats out of the way, you should know: the basics of user control are time-tested and well organized and in aws, generally speaking, the defaults work.

aws has already gone to great lengths to make sure that everything you do in the cloud is as secure as possible -- that is, everything is locked down by default. our job is to learn how to *unlock* down... or is it to lock *up*... or...

never mind.

## you already know a lot

at this point we have a lot of experience and hopefully a fair amount of comfort navigating the `linux` environment and the command line. in particular, you know about permission structures

In [ ]:
ll .

On my mac, I have a

+ user `zach.lamberty`
+ group `staff`
+ different permission structures for different files

On my `ec2`, I have a

+ user `ubuntu`
+ group `ubuntu`
+ different permission structures for different files

my mac user `zach.lamberty` and `ec2` user `ubuntu` is also a member of possibly multiple groups:

In [ ]:
%%sh
groups

this arrangement (having users, having users belong to groups, and having each file hold permissions on the user and group level) is fairly common and time-tested.

we have users, we assign users to multiple groups, and we grant read, write, and some other combination of list / execute permission to different resources on a per-user, per-group, or global basis

most of this carries over into `aws` in a service called `iam`

## semantics and relationships

permissions structures usually exist as *people* who belong to *groups* that can *do things*. the language changes from application to application, so let's lay down a couple of the main terms and the relationships between them:

1. single people and things ("things?", you ask) are **`users`**
2. collections of one or more **`users`** are called **`groups`**
3. **`users`** and **`groups`** of **`users`** can "take on" (aka temporarily step into and out of) **`roles`**
4. **`users`**, **`groups`**, and **`roles`** can possess **`permissions`** or **`policies`**
5. **`policies`** are collections of **`permissions`**

you're a sysad now, right?

maybe some more details will help.

open the [`iam` console](https://console.aws.amazon.com/iam/home?region=us-east-1) for your `aws` account

## users

a `user` is an individual account for your aws instance. although we typically think of a `user` as being a living person, the idea is generalized to include any independent entity which could do something (or be prevented from doing something):

1. people who log in to machines and do things
2. accounts you create to run specific programs (e.g. a database) or to do a specific job (e.g. run an ETL process)
3. system accounts that need special elevated privileges
4. other

the end goal is to give every user permission to do the things they need to do -- and *only* those permissions.

in our initial account setup, we created a user for our personal aws use. let's look through the properties of that user object.

1. open [the `iam` users dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/users) ("Users" in the left menu)
2. click the link in the "User name" column of the table

1. permissions tab
    1. the summary page starts off with the permission tab active
    2. permissions that our special user has are collected here as "policies" (more below).
    3. users can get permissions directly, or via group membership
    4. the “policy type” field will explain the source of a given permission policy.
    5. click the drop-down on any of the attached policies to see the full list of permissions this user has.
2. groups tab
    1. the "groups" tab displays all the groups to which your user belongs (and which permissions those `groups` grant you)
3. Tags tab
    1. this tab allows you to add "tags" to your user -- useful if you want to track information across users, add metadata, or group applications for billing based on the users involved
4. security credentials tab
    1. Access keys are created for API access to various services (see `aws cli` lecture)
5. access advisor tab
    1. for more advanced usage and creating access policies

**<div align="center">mini exercise: look at your user's access advisor</div>**

1. go to the `users` dashboard for the `iam` service
1. access your user (the one you log in as, probably your `guid`)
1. click on the "Access Advisor" tab
1. search for `ec2` in the "allowed services" text box
1. see the "Last accessed" section for information on when you last interacted with `ec2`

## groups

groups are simply collections of users (and possibly other groups). from a security perspective, these exist to be just a *bit* lazy.

we create `group`s as a way of saying "all these `user`s are basically the same in *this* way", where "*this* way" is a statement about their permissions.

we could go through the list of people in that group and update each person's permissions one at a time, but that would be very boring.

in the real world, these are usually permissions assigned to people based on business logic, especially employment position, for example

+ these people *are* data scientists
+ these people *are* in our DC office
+ these people *are* working on project X

you create a group for that identity (*e.g.* "data_scientists") and give the *group* the needed permissions. this means that

1. when you add a new data scientist to your company they get identical permissions to every other data scientist
1. when you need to grant or revoke permissions for data scientists, you can do it in one fell swoop

click now on [the `iam` groups dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/groups) ("groups" in the left menu).

this should bring up a table with your groups in them. click on one of your groups and you will land on a summary page for that group, which contains the following tabs:

1. users tab
    1. a list of the users in this group
    2. add or remove any number of users from this group -- note: this is instantaneous
2. permissions tab
    1. a list of the policies that are attached to this group (no drop-down like with users)
3. access advisor tab
    1. same as above for users -- advanced dialog for designing access policies

**<div align="center">mini exercise: look at your group's access advisor</div>**

1. go to the `groups` dashboard for the `iam` service
1. access your `admin` group (created in our first lecture when we set up billing)
1. click on the "Access Advisor" tab
1. search for `ec2` in the "filter" text box
1. see the "Last accessed" section for information on when you last interacted with `ec2` using a group policy

## roles

a role is similar to a group in that it is an abstract set of permissions that an `individual` could get. however, it is *different* than a `group`, in that it is something a `user` *becomes* and perhaps also *stops being*

+ a `user` *is* in a `group` (persistent)
+ a `user` can take on and then shed a `role` (temporary)

a metaphor: the position of `GU MAST professor` (a `role`) has certain privileges and responsibilities, but different people (individual `user`s) take turns stepping into that `role`.

while I am in my role as a `GU MAST professor`, I can do some special things that I could not do outside of that role. for example, I am allowed to assign you homework assignments about AWS services.

but I am not *always* in this role. if I started posting homework assignments next summer and demanded you complete them, it would not go well.

click now on [the `iam` roles dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/roles) ("Roles" in the left menu).

this should bring up a table with your roles listed there (you may have none). if you do, click on one of your roles and you will land on a summary page for that role, which contains the following tabs:

1. permissions tab
    1. a list of the policies that are attached to this group (no drop-down like with users)
1. trust relationships tab
    1. often organizations will have external user management systems (e.g. windows active directory). "trust relationships" are statements that what those external user systems say should be trusted
1. access advisor tab
    1. same as above for users -- advanced dialog for designing access policies
1. revoke sessions
    1. an emergency lever to restrict access to this role and its associated permissions

## policies

a policy is a collection of conceptually related permissions.

in the `linux` world, permission is set on the file level with a set of `allow`/`deny` indicators about read, write, or execute permissions. those permissions are collected into a `mode` (`rwx`). so we already know of times where we have "things" and we give "users" specific "permissions" to them.

in `aws`, everything is a `rest`ful api endpoint that is -- by default -- not accessible. we grant permissions to these endpoints to different `user`s, `group`s, or `role`s in a similar format.

there's a useful generalization in `aws`: a `policy` is a big bundle of permission grants or restrictions.

policies are generally given descriptive names like `AmazonS3ReadOnlyAccess`, and hold a definition of every permission needed to interact with a given "service" (a collection of `api`s for one generic purpose, like `ec2`).

for example, `AmazonS3ReadOnlyAccess` will grant all the permissions required to read files on `s3` but no further.

amazon cultivates many of these collections of permissions, and usually the ones you need are

1. named in an obvious way, and
2. added to the desired accounts as you set up the services

click now on [the `iam` policies dashboard](https://console.aws.amazon.com/iam/home?region=us-east-1#/policies) ("Policies" in the left menu).

this should bring up a table with thousands of policies (mostly created by AWS and not you). click on any one of them and you will land on a summary page for that policy, which contains the following tabs:

1. permissions tab
    1. unlike the previous tabs, this permissions tab lists real *permissions* as a `json` string. see below for more
2. policy usage
    1. the `user`s, `group`s, or `role`s that have this policy attached to them
3. policy versions
    1. a versioned (numbered) history of the permission json statement
4. access advisor tab
    1. same as above -- advanced dialog for designing access policies

## permissions

A permission is a codification of an access to or restriction from specific service. each permission consists of four pieces:

1. an aws service (e.g. ec2, s3, rds)
1. an access level (read, write, etc)
1. a resource
    1. if you have multiple resources (like hundreds of `s3` buckets) in your organization, this allows you indicate which instance(s) of those services this rule applies to
1. request condition
    1. a parameterizable prerequisite for exercising this permission

the tabular view on the "policy summary" button tab is a nice overview, but the real policy definition is the `json` blob on the second tab. for example, the `AmazonS3ReadOnlyAccess` policy has the following definition


```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:Get*",
                "s3:List*"
            ],
            "Resource": "*"
        }
    ]
}
```

you can read this policy block as saying: "whoever has this policy attached to it may execute any `Get` or `List` function on anything in `s3`"

**<div align="center">mini exercise: look at the billing admin policy we created</div>**

cast your mind wayyyyy back to lecture 1, in an exercise titled "really awful billing permission hack that sucks but now is the best time to do it"

1. go to the `policies` dashboard for the `iam` service
1. find the `billing_admin` policy we created (might have a different name, but this is what we recommended)
1. look at the policy `json` on the "permissions" tab

this policy should look roughly like

```json
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "Stmt1503972100000",
            "Effect": "Allow",
            "Action": [
                "aws-portal:*"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}
```

## putting it together

let's create a user, assign that user into a group, and attach meaningful policies to that group

**<div align="center">mini exercise: let's create a `gu511` user</div>**

1. go to the `users` dashboard for the `iam` service
1. create a user ("Add user")
    1. call this user `gu511`
    1. give this user *programmatic access only*
    1. create a group `ec2_s3_full_access`
        1. find the `AmazonEC2FullAccess` policy and add it
        1. find the `AmazonS3FullAccess` policy and add it
    1. put `gu511` in this group

## other

the remaining `iam` tabs are all useful in certain contexts, but we won't dive into them right now.

1. identity providers: allows you to "trust" outside resources for identity authentication
2. account settings: general security settings
3. access analyzer: create a monitor for certain access uses, and trigger responses
4. credential report: full automated export of "who" can do "what"
5. organization activity and service control policies: for checking all access activity of an external organization into your account, or from your account into someone else's (outside scope)

<strong><div align="center">UNLEASH THE POWER</div></strong>
<div align="center"><img width=600 src="https://cdn.dribbble.com/users/125948/screenshots/3818579/lock-variation-dribbble.png"></div>

# END OF LECTURE

next lecture: [AWS programmatic interfaces](009_cli_boto.ipynb)